In [21]:
from flask import Flask, request, jsonify, render_template,send_file
from flask_jwt_extended import JWTManager, create_access_token, jwt_required, get_jwt_identity
from flask_mysqldb import MySQL
import cv2 as cv
import base64
from PIL import Image
from io import BytesIO
import json


app = Flask(__name__)

# MySQL Configuration
app.config['MYSQL_HOST'] = 'localhost'
app.config['MYSQL_USER'] = 'prabhav'
app.config['MYSQL_PASSWORD'] = 'Prabhav123!'
app.config['MYSQL_DB'] = 'image_db'
app.config['JWT_SECRET_KEY'] = 'super-secret'
mysql = MySQL(app)
jwt = JWTManager(app)

@app.route('/')
def view():
    return render_template("index.html")

@app.route('/authentication_page')
def authentication_page():
    return render_template("authentication.html")

@app.route('/upload_page')
def upload_page():
    return render_template("upload1.html")

@app.route('/home_page')
def home_page():
    return render_template("homepage.html")

@app.route('/video_cus_page')
def video_cus_page():
    return render_template("video_cus.html")

# Route for user registration

@app.route('/signup', methods=['POST'])
def signup():
    cur = mysql.connection.cursor()
    username = request.json['username']
    name = request.json['name']
    email = request.json['email']
    password = request.json['password']
    print(username+name+email+password)
    cur.execute("INSERT INTO Users (Username, Name, Email, Password) VALUES (%s, %s, %s, %s)", (username, name, email, password))
    mysql.connection.commit()
    cur.close()
    return jsonify({"message": "User registered successfully"}), 200

# Route for user login
@app.route('/login', methods=['POST'])
def login():
    username = request.json['username']
    password = request.json['password']
    print("going to login"+username+password)
    cur = mysql.connection.cursor()
    result = cur.execute("SELECT * FROM Users WHERE Username = %s AND Password = %s", (username, password))
    if result > 0:
        print("it is somehow working")
        access_token = create_access_token(identity=username)
        return jsonify(access_token=access_token), 200
    else:
        return jsonify({"error": "Invalid username or password"}), 401

# Protected route
@app.route('/protected', methods=['POST'])
@jwt_required()
def protected():
    print('entering')
    current_user = get_jwt_identity()
    return jsonify(logged_in_as=current_user), 200

@app.route('/upload', methods=['POST'])
def upload():
    # print("hi")
    try:
        files = request.files.getlist('files')
        username = request.form['username']  # Parse JSON data sent from frontend
        # username = username['sub']
        print(username)
        # print(files)
        cur = mysql.connection.cursor()
        for file in files:
            name = file.filename
            image_bytes = file.read()
            extension = file.content_type
            # Convert the image bytes to a PIL Image object
            # image = Image.open(BytesIO(image_bytes))
            # Display the image
            # image.show()
            size = len(image_bytes)
            print(name , size , extension)
            print(type(image_bytes))
            # print(image_bytes)
            cur.execute("INSERT INTO images (username, image_blob, name, extension, size) VALUES (%s,%s, %s, %s, %s)",
                           (username,image_bytes, name, extension, size))
        # username = data.get('username')

        # print(username)
        # Connect to MySQL database

        # Commit the transaction
        mysql.connection.commit()
        cur.close()

        return jsonify({'message': 'Images uploaded successfully'}), 200

    except Exception as e:
        print(e)
        return jsonify({'error': str(e)}), 500

@app.route('/display_images')
def display_images():
    try:
        cur = mysql.connection.cursor()
        cur.execute("SELECT * FROM images")
        images = cur.fetchall()
        upload_images=[]
        for image in images:
            encoded_image = base64.b64encode(image[2]).decode('utf-8')
            upload_images.append(f"data:{image[4]};base64,{encoded_image}")

            print(image[3])
            print(image[4])
            print(image[2])

        cur.close()
        
        # Pass the images to the template for rendering
        return render_template('display_images.html', upload_images=upload_images)

    except Exception as e:
        print(e)
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [17/Mar/2024 00:12:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Mar/2024 00:12:33] "GET /Bard_Generated_Image(3).jpeg HTTP/1.1" 404 -
127.0.0.1 - - [17/Mar/2024 00:12:35] "GET /authentication_page HTTP/1.1" 200 -


In [ ]:
from flask import Flask, request, jsonify, Response, send_file, render_template
from flask_mysqldb import MySQL
import cv2
import numpy as np
import base64
from io import BytesIO

app = Flask(__name__)

# MySQL Configuration
app.config['MYSQL_HOST'] = 'localhost'
app.config['MYSQL_USER'] = 'prabhav'
app.config['MYSQL_PASSWORD'] = 'Prabhav123!'
app.config['MYSQL_DB'] = 'image_db'
mysql = MySQL(app)

# Route to generate and serve the video
@app.route('/')
def video_player():
    return render_template('video.html')

@app.route('/generate_video')
def generate_video():
    # Connect to the database
    cur = mysql.connection.cursor()

    # Retrieve image data from the database
    cur.execute("SELECT * FROM images")
    rows = cur.fetchall()

    # Create a VideoWriter object
    out = cv2.VideoWriter('output.avi', cv2.VideoWriter_fourcc(*'DIVX'), 1, (640, 480))

    # Iterate over the rows and add each image to the video
    for row in rows:
        try:
            # Assuming the image data is in the second column
            image_data = row[2]
            # Convert the image data to a numpy array
            nparr = np.frombuffer(image_data, np.uint8)
            # Decode the image
            img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
            # Resize the image to fit the video frame size
            img = cv2.resize(img, (640, 480))
            # Write the image to the video
            out.write(img)
        except Exception as e:
            print(f"Error processing image: {e}")

    # Release the VideoWriter object
    out.release()

    # Close the database connection
    cur.close()

    # Return the generated video file
    return send_file('output.avi', mimetype='video/avi')

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [16/Mar/2024 23:39:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Mar/2024 23:40:20] "GET / HTTP/1.1" 200 -
